# Preparing Libraries and Installations


In [1]:
%pip install bs4 selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/484.6 kB ? eta -:--:--
   ---------------------- ----------------- 276.5/484.6 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 484.6/484.6 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ---------------------------------------- 307.7/307.7 kB 9.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    StaleElementReferenceException,
)


from bs4 import BeautifulSoup

from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

import pandas as pd
import time

# Scraping the Website

In [17]:
# URL Ulasan dari Female Daily
base_url = "https://reviews.femaledaily.com/products/cleanser/toner/for-skin-s-sake-fss/glycolic-acid-gel-10-with-tranexamic-acid?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page={}"
page = 1

# Koneksi ke mongodb
username = "ayukrisna"
password = "bigdata"
host = "ayukrisna.c81wxan.mongodb.net"
dbname = "femaledaily_test"
uri = f"mongodb+srv://{username}:{password}@{host}/{dbname}?retryWrites=true&w=majority"

try:
    client = MongoClient(uri)
    db = client[dbname]
    collection = db['test_review']

    print("Successfully connected to MongoDB!")
except ConnectionFailure as e:
    print(f"Failed to connect to MongoDB: {e}")
    exit()

try:
    # Inisialisasi option chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Inisialisasi WebDriver
    service = Service(executable_path = ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    #data frame
    df_data = pd.DataFrame(columns = ['age', 'skin_type', 'rating', 'recommended', 'review','usage_period', 'purchase_point'])

    # Scraping data
    while True:
        # Buka URL
        url = base_url.format(page)
        driver.get(url)
        print(page)
        wait = WebDriverWait(driver, 10)

        # Ekstrak dan print title
        # print(driver.title)
        time.sleep(5) #tunggu 5 detik

        # Get List Reviews
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        list_reviews = soup.find(class_='list-reviews')
        reviews = list_reviews.find_all(class_='review-card')

        # if not list_reviews:
        #     print("No reviews found on this page.")
                       
        for review in reviews:
            #left column
            left_col = review.find(class_= 'review-card-col left')
            age = left_col.find('p', class_='profile-age').get_text(strip=True)
            skin_type = left_col.find('p', class_='profile-description').get_text(strip=True)

            #right column
            right_col = review.find(class_= 'review-card-col right')
            wrapper = right_col.find(class_='review-content-wrapper')

            #start ratings
            star_icons = wrapper.find_all('i', class_='icon-ic_big_star_full')
            rating = len(star_icons)

            #recommend
            recommendation_tag = wrapper.find('p', class_='recommend')
            if recommendation_tag:
                if 'recommends' in recommendation_tag.get_text():
                    recommended = "recommended"
                else:
                    recommended =  "not recommended"
            else:
                recommended =  "unknown"

            #text review
            try:
                review_text = wrapper.find(class_='text-content').get_text(strip=True)
            except  (StaleElementReferenceException, NoSuchElementException):
                review_text = ""

            #usage period and purchase point
            info_wrapper = wrapper.find(class_='information-wrapper')
            usage_period = info_wrapper.find('span', string='Usage Period ').find_next('b').get_text(strip=True)
            purchase_point = info_wrapper.find('span', string='Purchase Point ').find_next('b').get_text(strip=True)
            
            # simpan ke frame
            df_data.loc[len(df_data)] = [age, skin_type, rating, recommended, review_text, usage_period, purchase_point]

            #simpan ke mongodb
            review_data = {
                'age': age,
                'skin_type': skin_type,
                'rating': rating,
                'recommended': recommended,
                'review': review_text,
                'usage_period': usage_period,
                'purchase_point': purchase_point
            }
            collection.insert_one(review_data)

        # Memeriksa apakah ada button next page
        try:
            next_button = driver.find_element(By.ID, 'id_next_page')
            if 'paging-prev-text-inactive' in next_button.get_attribute('class'):
                break
        except NoSuchElementException:
            break

        page += 1
        time.sleep(2)

#exception
except WebDriverException as e:
    print(f"WebDriverException: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
except TimeoutException:
    print("The element was not clickable within the given time")
finally:
    driver.quit()

pd.set_option('display.max_colwidth', None)
df_data.head()

Successfully connected to MongoDB!
1
2
3


,age,skin_type,rating,recommended,review,usage_period,purchase_point
0,19 - 24,"Dry, Medium, Warm",5,recommended,"untuk glycolic acid sebelumnya aku pake cosrx yg aha 7 whiteheads trs ganti jadi ini dan ga berasa cekat cekit sama sekali padahal 10%, aku pake ini sebagai serum exfo dan tonernya pake 1004 mild exfoliating, pagipagi kulit berasa lebih plump dan alus suka bgt sih tiap kali pake ini👍👍",1 week - 1 month,Shopee
1,19 - 24,"Oily, Light, Neutral",5,recommended,"Dari jaman belajar skincare thn 2018 aku ud pk fss yg hyaluronic, vit c dan retinol (sblm reformulasi) itu semua cocok di aku dn buat jerawat aku jauh lbh mendingan. Makanya aku tertarik nyoba klo fss ngeluarin product baru 😂 Aku ud pake ini rutin sekitar 5 bulanan lah seminggu 2x sumpah pelan2 kelihatan hasilnya jerawat jd jarang muncul trus yg aku suka klo pake ini malem2, pagi2 bangun kulitnya kenceng bgtt bouncy trus jd soft. Dia itu texturenya watery gel bening lumayan cepet kering. Biasanya aku cuman pk 2 pump ajaa sbg serum. Dia mmg agak cekat cekit dikit di muka cuman masih d batas wajar si buat aku. Klo aku pk ini biasanya aku paduin sm skincare yg calming kek centella gt gk aku combine pk bahan aktif lain kec niacinamide. Ini product bener2 underated pdhl bagus bgtt! 💕😭🫶 cocok bgt klo yg mau coba upgrade dr product2 eksfoliasi yg terlalu mild.",3 months - 6 months,Shopee
2,25 - 29,"Combination, Medium Dark, Warm",5,recommended,"Aku kaget banget liat exfo gel semurah ini!!! Klo diliat harganya sama kyk exfo merk sebelah. Tapi!!! Ini 100ml loh!!! Aku kaget banget saat barangnya datang ternyata gede banget!!! Dan super duper worth it! Km bisa pake exfo semua bagian badan juga klo segini gedenya! Bisa buat ketiak, siku, lutut, punggung.exfo ini juga bagus banget buat km yg males banget rutinin facial. Krn menurutku pake ini, komedo itu jadi lunak, gampang keluar. Aduhhh the best ini mah. Btw ini perih di aku, tapi malah perihnya ini sih yg bikin nagih!!! Wkwkwkwk (klo newbie exfo mending pake yg mandelic/lactic aja sih)",1 month - 3 months,XBeauty by Female Daily
3,18 and Under,"Combination, Medium Dark, Neutral",5,recommended,"JUJUR inituh jarang bangeeeet yang bahasss, bahkan aku taunya juga dari review di fd ini! sebelumnya emang kulitku badak banget si, pake exfo apapun gamempan, tapi waktu pake ini bener-bener works bangettt!! kulitku berasa alus (pelan-pelan sih ak pake udah 2-3 bulanan), happy banget!!!",1 month - 3 months,Brand website
4,25 - 29,"Combination, Medium, Neutral",5,recommended,"serum exfoliasi paling bangus di muka aku sangat bekerja dengan baik, teksture gel yang gak lengket, pasti ada efek clekit clekit itu tandanya product sedang bekerja dengan baik, percaya deh besok pagi muka jadi kenceng, glowing, udah kayak ubin kinclong dan mulus",1 month - 3 months,sociolla
